In [1]:
# We always start with a dataset to train on. Let's download the tiny shakespeare dataset
# !wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-01-16 18:11:21--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.03s   

2024-01-16 18:11:21 (34.0 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [1]:
# read it in to inspect it
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [2]:
print(f"length of the text file = {len(text)}")

length of the text file = 1115394


In [3]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [4]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [5]:
stoi = {s:i for i, s in enumerate(chars)}
itos = {i:s for s, i in stoi.items()} 
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

In [6]:
print(encode('hi there'))
print(decode(encode('hi there')))

[46, 47, 1, 58, 46, 43, 56, 43]
hi there


In [7]:
import torch 
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [8]:
n = int(len(data) * 0.9) 
train_data = data[:n]
val_data = data[n:]

In [9]:
block_size = 8 
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [10]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"input: {context} ---> target: {target}")

input: tensor([18]) ---> target: 47
input: tensor([18, 47]) ---> target: 56
input: tensor([18, 47, 56]) ---> target: 57
input: tensor([18, 47, 56, 57]) ---> target: 58
input: tensor([18, 47, 56, 57, 58]) ---> target: 1
input: tensor([18, 47, 56, 57, 58,  1]) ---> target: 15
input: tensor([18, 47, 56, 57, 58,  1, 15]) ---> target: 47
input: tensor([18, 47, 56, 57, 58,  1, 15, 47]) ---> target: 58


In [18]:
ix = torch.randint(len(data) - block_size, (batch_size, ))
x = torch.stack([data[i:i+block_size] for i in ix])
y = torch.stack([data[i+1:i+block_size+1] for i in ix])
print(ix)
print(x)
print(y)

tensor([459831, 425370, 157902, 365622])
tensor([[47, 42, 39, 51, 43,  6,  0, 32],
        [53, 52, 42,  6,  1, 58, 46, 39],
        [53, 59, 56,  1, 61, 47, 58, 46],
        [43, 50, 43, 63,  8,  0, 21,  1]])
tensor([[42, 39, 51, 43,  6,  0, 32, 46],
        [52, 42,  6,  1, 58, 46, 39, 58],
        [59, 56,  1, 61, 47, 58, 46,  1],
        [50, 43, 63,  8,  0, 21,  1, 57]])


In [21]:
torch.manual_seed(1337)
batch_size = 4 # how many examples to put into the model in parellel 
block_size = 8 # the maximum context length to predict the next token  

def get_batch(split):
    data = train_data if split=='train' else val_data 
    ix = torch.randint(len(data) - block_size, (batch_size, ))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y 

xb, yb = get_batch('train')
print('inputs')
print(xb)
print(f'shape = {xb.shape}')
print('outputs')
print(yb)
print(f'shape = {yb.shape}')
print('----------------------------')

for bi in range(batch_size):
    for ti in range(block_size):
        context = xb[bi, :ti+1]
        target = yb[bi, ti]
        print(f"inputs: {context} --> target: {target}")

inputs
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
shape = torch.Size([4, 8])
outputs
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
shape = torch.Size([4, 8])
----------------------------
inputs: tensor([24]) --> target: 43
inputs: tensor([24, 43]) --> target: 58
inputs: tensor([24, 43, 58]) --> target: 5
inputs: tensor([24, 43, 58,  5]) --> target: 57
inputs: tensor([24, 43, 58,  5, 57]) --> target: 1
inputs: tensor([24, 43, 58,  5, 57,  1]) --> target: 46
inputs: tensor([24, 43, 58,  5, 57,  1, 46]) --> target: 43
inputs: tensor([24, 43, 58,  5, 57,  1, 46, 43]) --> target: 39
inputs: tensor([44]) --> target: 53
inputs: tensor([44, 53]) --> target: 56
inputs: tensor([44, 53, 56]) --> target: 1
inputs: tensor([44, 53, 56,  1]) --> target: 58

In [25]:
import torch 
import torch.nn as nn 
import torch.nn.functional as F 
torch.manual_seed(2024)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None): # targets = (B, T)
        logits = self.token_embedding_table(idx) # idx = (B, T)

        if targets is None:
            loss = None 
        else:
            B, T, C = logits.shape 
            logits = logits.view(B*T, C) 
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss  

    def generate(self, idx, max_new_tokens):
        for i in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:, -1, :] # (B, C) 
            probs = F.softmax(logits, -1) # (B, C)
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx  

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx=torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

torch.Size([32, 65])
tensor(4.4547, grad_fn=<NllLossBackward0>)

Yr.YjvmHFQqBWq$?lP$PbdNM.
IT:aT:XAFhQMX:WMlcvTqeVtA:$?whM&KAXBk$JBh,zo!-fHP?$quhdMrn yNx-Vi3rXf?:Hxw


In [26]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [28]:
print(sum([p.nelement() for p in m.parameters()]))

4225


In [30]:
batch_size = 32 

for steps in range(100):
    xb, yb = get_batch('train')

    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step() 

print(loss.item())

4.33015775680542
